# LambdaCat Walkthrough

1. Standard small categories (Terminal, Discrete, Δ³, Walking Isomorphism)
2. Law checks for categories (identity, associativity)
3. Functors: building and verifying laws
4. Natural transformations: components and naturality squares
5. Category operations: opposite category
6. Monads and Kleisli arrows (Id, Maybe)
7. Visualization: Mermaid diagrams for categories, functors, naturality



In [ ]:
from LambdaCat.core.standard import terminal_category, discrete, simplex, walking_isomorphism
from LambdaCat.core.laws import run_suite
from LambdaCat.core.laws_category import CATEGORY_SUITE

Term = terminal_category()
Disc = discrete(["A","B"]) 
Delta3 = simplex(3)
Iso = walking_isomorphism()

for C in (Term, Disc, Delta3, Iso):
    assert run_suite(C, CATEGORY_SUITE).ok

Term, Disc, Delta3, Iso


## Functors
We build a functor `F: Δ³ → Iso` that maps objects and morphisms, then verify the functor law suite.


In [ ]:
from LambdaCat.core.functor import FunctorBuilder
from LambdaCat.core.laws_functor import FUNCTOR_SUITE

F = (FunctorBuilder('F', source=Delta3, target=Iso)
     .on_objects({"0":"A","1":"A","2":"B","3":"B"})
     .on_morphisms({"0->1":"id:A","1->2":"f","2->3":"id:B","0->3":"f"})
     .build())

report = run_suite(F, FUNCTOR_SUITE)
print(report.to_text())


## Natural transformations
Define η: F ⇒ F with components on objects, verify component typing and commuting squares.


In [ ]:
from LambdaCat.core.natural import Natural
from LambdaCat.core import NATURAL_SUITE

eta = Natural(source=F, target=F, components={"0":"id:A","1":"id:A","2":"id:B","3":"id:B"})
print(run_suite(eta, NATURAL_SUITE).to_text())


In [ ]:
# Violation demo: wrong-typed component for object '1'
bad = Natural(source=eta.source, target=eta.target, components={**eta.components, "1": "f"})
print(run_suite(bad, NATURAL_SUITE).to_text())


## Opposite category
Construct `C^op` for Δ³ to examine dual arrows.


In [ ]:
from LambdaCat.core.ops_category import opposite_category
Delta3_op = opposite_category(Delta3)
len(Delta3_op.arrows), Delta3_op.arrows[0].name


## Monads and Kleisli
Compose effectful computations with `Kleisli` for `Maybe` and `Id`.


In [ ]:
from LambdaCat.core.fp.kleisli import Kleisli, kleisli_identity
from LambdaCat.core.fp.instances.maybe import Maybe
from LambdaCat.core.fp.instances.identity import Id

parse_int = Kleisli(lambda s: Maybe(int(s)) if str(s).isdigit() else Maybe(None))
recip = Kleisli(lambda n: Maybe(None) if n == 0 else Maybe(1.0 / n))
pipe = parse_int.then(recip)
print(pipe.run("12"), pipe.run("oops"))

I = kleisli_identity(Id.pure)
inc = Kleisli(lambda n: Id(n + 1))
dbl = Kleisli(lambda n: Id(n * 2))
print(I.run(5), inc.then(dbl).run(3))


## Visualization
Render Mermaid diagrams for categories, functors, and naturality (paste into Markdown or tools that support Mermaid).


In [ ]:
from LambdaCat.extras.viz_mermaid import render_all
md_map = render_all({'Delta3': Delta3, 'Iso': Iso, 'F': F, 'eta': eta}, out_dir=None)
print(list(md_map.keys()))
print(md_map['Delta3__category.md'][:200])


## Functor composition
Construct another functor `G: Iso → Iso` and verify that the composed functor `H = G ∘ F` satisfies the functor laws.


In [ ]:
from LambdaCat.core.functor import CatFunctor
# Identity-like endofunctor on Iso
G = CatFunctor(
    name='G', source=Iso, target=Iso,
    object_map={'A':'A','B':'B'},
    morphism_map={'id:A':'id:A','id:B':'id:B','f':'f','g':'g'}
)
# Compose by mapping F’s object/morphism images through G
H = CatFunctor(
    name='H', source=Delta3, target=Iso,
    object_map={X: G.object_map[Y] for X,Y in F.object_map.items()},
    morphism_map={m: G.morphism_map[n] for m,n in F.morphism_map.items()},
)
print(run_suite(H, FUNCTOR_SUITE).to_text())


## Endofunctors and fixed points (sketch)
Explore simple endofunctors on Δ³ and observe their effect on object/morphism maps.


In [ ]:
F2 = CatFunctor(
    name='F2', source=Delta3, target=Delta3,
    object_map={'0':'0','1':'1','2':'2','3':'3'},
    morphism_map={'id:0':'id:0','id:1':'id:1','id:2':'id:2','id:3':'id:3','0->1':'0->1','1->2':'1->2','2->3':'2->3','0->2':'0->2','1->3':'1->3','0->3':'0->3'}
)
print('F2 objects:', F2.object_map)
print('F2 morphisms (subset):', {k: F2.morphism_map[k] for k in ['0->1','1->2','2->3']})


## Advanced: Writer and State monads
Demonstrate accumulation (Writer) and threaded state (State) with small examples; relate to algebraic effects used in proof assistants.


In [ ]:
from LambdaCat.core.fp.instances.writer import Writer
from LambdaCat.core.fp.typeclasses import Monoid

class ListMonoid(Monoid[list[str]]):
    def empty(self) -> list[str]:
        return []
    def combine(self, a: list[str], b: list[str]) -> list[str]:
        return a + b

W0 = Writer.pure(1, ListMonoid())
W1 = W0.map(lambda x: x + 1).tell(["inc"]).map(lambda x: x * 2).tell(["dbl"]) 
print(W1.value, W1.log)

from LambdaCat.core.fp.instances.state import State

def push(x: int) -> State[list[int], None]:
    return State(lambda st: (None, st + [x]))

prog = State.pure(None).bind(lambda _: push(1)).bind(lambda _: push(2)).bind(lambda _: push(3))
print(prog.run([]))


## Advanced: Agent-aligned category examples (no runtime)
Model reversible transformations via functors into the walking isomorphism; sketch how these correspond to invertible steps in pipelines.


In [ ]:
from LambdaCat.core.functor import FunctorBuilder
# Functor mapping an abstract two-step flow (0->1->2) to an isomorphism
D2 = simplex(2)
Fiso = (FunctorBuilder('Fiso', source=D2, target=Iso)
        .on_objects({'0':'A','1':'B','2':'A'})
        .on_morphisms({'0->1':'f','1->2':'g','0->2':'id:A'})
        .build())
print(run_suite(Fiso, FUNCTOR_SUITE).to_text())
